In [ ]:
# import numpy as np
# import pandas as pd
# import json

# ## SET PATH to code directory in the project folder
# code_path = 'C:\\evdoxiataka\\effi_user_study1\\code'
# import sys  
# sys.path.insert(1, code_path)

# ## SET folder to project directory
# folder="C:\\evdoxiataka\\effi_user_study1\\"

# from utils.utils import *
# from utils.training import oneoff_training_evaluation, iml_training_evaluation
# from utils.visualization import *

# import matplotlib
# import matplotlib.pyplot as plt

# import warnings
# from pandas.errors import SettingWithCopyWarning
# warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# ## Any of the training set's attributes could be added here, protected or not, sensitive or not.
# ## We evaluated fairness for the following attributes
# sensitive_attrs = ["CODE_GENDER",
#                    "NAME_FAMILY_STATUS",
#                    "AGE",
#                    "REGION_RATING_CLIENT",
#                    "REGION_RATING_CLIENT_W_CITY",
#                    "OBS_30_CNT_SOCIAL_CIRCLE",
#                    "DEF_30_CNT_SOCIAL_CIRCLE",
#                    "OBS_60_CNT_SOCIAL_CIRCLE",
#                    "DEF_60_CNT_SOCIAL_CIRCLE"]
# fs = 0.5

# colors = ['r','g','b','orange','purple', 'y','cyan','magenta','tab:blue']

import numpy as np
import pandas as pd
import ast

########################SET folder to path of code directory in the project folder#####################
## SET PATH to code directory in the project folder
code_path = "C:\\evdoxiataka\\effi_user_study1\\code"
import sys  
sys.path.insert(1, code_path)

########################SET folder to project directory path#####################
folder= "C:\\evdoxiataka\\effi_user_study1\\"

from utils.data_loading import get_feedback_df, get_training
from utils.utils import attributes_names_mapping
from utils.training import oneoff_training_evaluation, iml_training_evaluation

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

sensitive_attrs = list(attributes_names_mapping.keys())
fs = 0.5

## Data loading

In [ ]:
## load training and test set
train_df_train = pd.read_csv(folder+'data/processed_data/train_df_train.csv', delimiter=',')
train_df_test = pd.read_csv(folder+'data/processed_data/train_df_test.csv', delimiter=',')

X_train_original = train_df_train.loc[:, ~train_df_train.columns.isin(['TARGET', 'SK_ID_CURR'])]
y_train_original = train_df_train.loc[:, train_df_train.columns == 'TARGET']
X_test_original = train_df_test.loc[:, ~train_df_test.columns.isin(['TARGET'])] ## dataset used for accuracy
y_test_original = train_df_test.loc[:, train_df_test.columns == 'TARGET']

## load original test set to be used for fairness metrics
train_df = pd.read_csv(folder+'data/processed_data/train_df.csv', delimiter=',')
train_df_test_or = train_df[train_df['SK_ID_CURR'].isin(train_df_test['SK_ID_CURR'].tolist())]
lst = train_df_test['SK_ID_CURR'].tolist()
train_df_test_or = train_df_test_or.reset_index().set_index('SK_ID_CURR').loc[lst].reset_index().set_index('index').rename_axis(None)
manipulate_categ_values(train_df_test_or)
train_df_test_bin = binning(train_df_test_or, train_df_test)

## load application data shown to participants through the UI
applications_df = pd.read_csv(folder+"data/processed_data/Applications_Converted.csv", index_col=0)

## load test data used to predict labels before shown to participants
test_df = pd.read_csv(folder+'data/processed_data/test_df_final.csv', delimiter=',')
# test_df = test_df.loc[:, ~test_df.columns.isin(['SK_ID_CURR'])]

## load participants' feedback
feedback_df = pd.read_csv(folder+'data/processed_data/Feedback_final.csv', delimiter=',', index_col=0)
feedback_df.drop(columns = "Lower bound",inplace = True)
feedback_df.drop(columns = "Upper bound",inplace = True)
feedback_df = feedback_df[feedback_df["Function"] == "select_reject_application"]

## get list of weights before feedback integration in right order
features_in_trainingOrder = ['Number of children', 'Income', 'Loan Credit amount ', 'Loan annuity',
       'Goods Price', "Region's Normalized Population", 'Age',
       'Years in current employment', 'Years since changing registration', 'Years since changing  identity document',
       'Has Mobile', 'Has Employee Phone', 'Has Work Phone', 'Mobile reachable',
       'Has Phone', 'Has email', 'Number of family members', "Region's Rating",
       'Region & City Rating', 'Application Hour',
       'Contact address located in Registration region', 'Work address located in Registration region',
       'Contact address located in work region', 'Contact city in Registration city',
       'Work address in Registration city', 'Contact city in work city',
       'Number of times social circle at default risk for 30 days', 'Number of times social circle defaulted on loan for 30 days',
       'Number of times social circle were at risk to default on loan for 60 days', 'Number of times social circle defaulted on loan for 60 days',
       'Years since changing phone', 'Number of Credit Bureau enquiries 1 hour before application',
       'Number of Credit Bureau enquiries 1 day before application', 'Number of Credit Bureau enquiries 1 week before application',
       'Number of Credit Bureau enquiries 1 month before application', 'Number of Credit Bureau enquiries 3 month before application',
       'Number of Credit Bureau enquiries 1 year before application', 'Installments', 'Gender',
       'Owns Car', 'Owns Property', 'Accompanied while applying',
       'Income type', 'Highest education level',
       'Family status', 'Housing situation', 'Occupation Type',
       'Application Day', 'Employer organization']
feature_weights_dict = {}
for i in range(49):
    feature_weights_dict[applications_df['ft_name_'+str(i)].tolist()[0]] = applications_df['ft_weight_'+str(i)].tolist()[0]
feature_weights = []
for feat in features_in_trainingOrder:
    feature_weights.append(feature_weights_dict[feat])

## convert string values to list of dicts
for i in feedback_df.index :
    s = feedback_df['Value'].loc[i]
    # print(i,feedback_df['App ID'].loc[i])
    s = s.replace("'", "\"")
    s = s.replace("Region\"s Rating", "Region\'s Rating")
    s = s.replace("Region\"s Normalized Population", "Region\'s Normalized Population")
    feedback_df.loc[[i], "Value"] = pd.Series([json.loads(s)], index=[i])

## keep only feature weights relevant to each APP ID
all_new_values = []
for i in feedback_df.index :
    app_id = feedback_df['App ID'].loc[i]
    value = feedback_df['Value'].loc[i]
    # new_value = {}
    new_value = feature_weights.copy()
    if value:
        for v_dict in value:
            _idx = v_dict['attribute'].index('_')
            if app_id == np.int64(v_dict['attribute'][0:_idx]):
                # print(new_value, v_dict['attribute'][(_idx+1):], np.float64(v_dict['value']))
                attr = v_dict['attribute'][(_idx+1):]
                new_value[features_in_trainingOrder.index(attr)]=np.float64(v_dict['value'])
                # new_value[]=np.float64(v_dict['value'])
    else:
        new_value = []
    ## normalize
    new_value = [v/sum(new_value) for v in new_value]
    # if sum(new_value) >1.0:
    #     print(sum(new_value), max(new_value),min(new_value))
    all_new_values.append(new_value)    
feedback_df['Value'] = all_new_values

## TRAINING

### Global Model - Labels ('fair'+'unfair')
The XGboost classifier is retrained. The feedback instances of all participants are added into the training set all at once.

**All the 'fair' and 'unfair'** labelled instances by participants are used. 

We set fs = 0.5.

In [ ]:
oneoff_training_evaluation(X_train_original, y_train_original,
                            X_test_original, y_test_original,
                           train_df_test_bin, test_df, applications_df,
                            sensitive_attrs, fs, feedback_df,
                           False, False,
                           'global-Labels', folder+"data/results/global/Labels/")

### Global Model - Labels ('unfair')
The XGboost classifier is retrained. The feedback instances of all participants are added into the training set all at once.

**Only the 'unfair'** labelled instances by participants are used. 

We set fs = 0.5.

In [ ]:
oneoff_training_evaluation(X_train_original, y_train_original,
                            X_test_original, y_test_original,
                           train_df_test_bin, test_df, applications_df,
                            sensitive_attrs, fs, feedback_df,
                           True, False,
                           'global-Labels_Unfair', folder+"data/results/global/Labels_Unfair/")

### Global Model - Labels+Weights ('fair'+'unfair')
The XGboost classifier is retrained. The feedback instances of all participants are added into the training set all at once.

**All the 'fair' and 'unfair'** labelled instances by participants **with proposed weight changes** are used. 

We set fs = 0.5.

In [ ]:
oneoff_training_evaluation(X_train_original, y_train_original,
                            X_test_original, y_test_original,
                           train_df_test_bin, test_df, applications_df,
                            sensitive_attrs, fs, feedback_df,
                           False, True, 
                           'global-Labels+Weights', folder+"data/results/global/Labels+Weights/")

### Personalized Models - Labels ('fair'+'unfair')
58 XGboost classifiers will be trained, one for each participant independently. The feedback instances of each participant will be added to the training set incrementally, one at a time, in increasing timestamps (the previous feedback instances of a participant remain in the training set as we add a new one). The XGboost classifier is retrained from scratch in every incremental step (iteration). 

**All the 'fair' and 'unfair'** labelled instances by participants are used. 

We set fs = 0.5.

In [ ]:
iml_training_evaluation(X_train_original, y_train_original,
                        X_test_original, y_test_original,
                        train_df_test_bin, test_df, applications_df,
                        sensitive_attrs, fs, feedback_df,
                        False, False, 
                        'personalized-Labels', folder+"data/results/personalized/Labels/")

### Personalized Models - Labels ('unfair')
58 XGboost classifiers will be trained, one for each participant independently. The feedback instances of each participant will be added to the training set incrementally, one at a time, in increasing timestamps (the previous feedback instances of a participant remain in the training set as we add a new one). The XGboost classifier is retrained from scratch in every incremental step (iteration). 

**Only the 'unfair'** labelled instances by participants are used.

We set fs = 0.5.

In [ ]:
iml_training_evaluation(X_train_original, y_train_original,
                        X_test_original, y_test_original,
                        train_df_test_bin, test_df, applications_df,
                        sensitive_attrs, fs, feedback_df,
                        True, False, 
                        'personalized-Labels_Unfair', folder+"data/results/personalized/Labels_Unfair/")

### Personalized Models - Labels+Weights ('fair'+'unfair')
The feedback instances of each participant will be added to the training set incrementally, one at a time, in increasing timestamps (the previous feedback instances of a participant remain in the training set as we add a new one). The XGboost classifier is retrained from scratch in every incremental step (iteration).

**All the 'fair' and 'unfair'** labelled instances with proposed weight changes are used. Participants weights of each instance are used to set feature weights of the training.

We set fs = 0.5.

In [ ]:
iml_training_evaluation(X_train_original, y_train_original,
                        X_test_original, y_test_original,
                        train_df_test_bin, test_df, applications_df,
                        sensitive_attrs, fs, feedback_df,
                        False, True, 
                        'personalized-Labels+Weights', folder+"data/results/personalized/Labels+Weights/")